# Notebook content overview

This notebook contains useful ifnormation to navigate the database of functional data hosetd on DataJoint. In particular it offers useful information to idenfiy relevant stimuli and actbvity inormation for a cell. In addition, it presents code that allows to extract the data related to the activity of the functionally matched neurons that is then useful to both carry out an analysis of correlation of activity between connected cells and to ideally extract the recpetive field of each functionally matched neuron.

### Definition of columns and imaging terms

This sections contains a breif overview and definition of what some of teh columns in the functional dataset stand for and their meaning in the world of medical imaging

### Stimuli

This section contains part of teh turtorial (with some additional clarifications I have hadded) provided by the MICrONS team to understand what each stimuli shown to the mouse is and how to eaxtrct the relevant stimuli picture and its corresponding activity for any of the functionally matched neuron.

### Spike Traces

This section contains part of the tutorial by teh MICrONS team to extract the spike trace and fluorescent trace of a cell and match it with the corresponding stimulus shown

### Selecting relevant stimuli for our analysis

**Identifying and extacting oracle stimuli**:

This section focuses on extracting the relevant activity data for stimuli of teh Oracle type. These are stimuli that are exactly identical across repeats and that have been erepeated a series of times. It should thus be usfeful to compare the activity across different neurons. I define a function to extract the indices in the shown mobvie stimuli at which these oracle stimuli occurr and then I also do a manual check to see if these stimuli are indeed the same.

**Extracting and saving relevant spike traces for Oracle stimuli and reliability threshold**:

In this section I define a series of functions that help me extract for each functionally matched cell either teh spike trace or the fluorescent trace of their activity in response to each of the 10 repeats of the oracle stimuli they where shown (and also the average activity). I thenn proceed to us ethe function to extract these activities annd save them.

    NOTE: This section imports the clean_functional_locations.csv** file from the neuron_depth.ipynb notebook (but you could do this also with a file containing the functional_features of the functionnal cells) and allows to save locally the ifnormation above in a pickle file called **clean_functional_ccmax_ost.pkl** if spike traces or **clean_functional_ccmax_calcium.pkl** if calcium traces.

### Extracting Relevant Movie Stimulus section

In this section I extract pickle files that contain the relevant frames of a single repeat of the Oracle stimulus shown to neurons and for which I have extracted the activity above. Since the oracle stimulus shown changes depending on the session and scan_idx the neuron was part of multiple files are extracted by the code present in this section.

    NOTE: This information has been saved locally has 15 files each named according to the following naming schema  m_{session}_{scan_idx}.pkl







# Imports

In [3]:
import random
random.seed(4)
import datajoint as dj
#from phase3 import nda, func, utils
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns
from PIL import Image
from matplotlib import cm
from collections import defaultdict
from scipy.optimize import curve_fit
import time
import gc
from PIL import Image
import warnings
import pickle

In [ ]:
dj.ERD(nda) # View schema ERD

# Definitions of columns and imaging terms

**session**: Think of this as the day the scan was taken in, since it might take multiple days to complete the whole imaging process

**scan_id**: This was the subvolume of the whole volume of the brain that was scanned in the specific two photon functional imaging session (multiple scans can be taken in the same session, ex. on day (session) 1 I scanned subvolume 1 and 2 on day (session) 2 I scanned 3 and 4...). (Think of forming a stack of sheets of papers, each being an image of the brain volume we are focusing on).

**imaging plane**: This is an even thinner 'sheet' of volume within the subvolume contained in the scan. Each scan is composed of multiple imaging planes. (Think of taking only a smaller volume of sheets of the full stack of the sheets of paper)

**field**: Within an imaging plane they divided the plan in different sections (so imagine a paper sheet that is in 4 pieces or 2 pieces or x pieces) and each of these sections was a field for which they took an image to derive calcium trace... (when then they talk about 'channels' in the functional specifications document they just mean the amount of resources they had to visualise that image at the time (ex. if they had two microscopes to visualise that image at the time it would be two channels, in this case they only had 1 so its always 1 channel)).

# Stimuli

The movie shown to the mouse for each scan is housed in `nda.Stimulus`. The movie is synced to the timestamps in `nda.FrameTimes`.

In [ ]:
scan_key = {'session': 4, 'scan_idx': 7}

First we choose the scan key and session that once joined with the Stimulus table will give us the movie show to the mouse in that session and scan

In [ ]:
fig, axs = plt.subplots(1, 2, dpi=150) # view first and last frame of movie slice
axs[0].imshow(mov1[:,:,60], cmap='gray')
axs[0].set_title(f'frame: m1')
axs[1].imshow(mov2[:,:,60], cmap='gray')
axs[1].set_title(f'frame: m2')

Then we fetch the movie from this matched table if we want to inspect it at any time or use it for other things

In [ ]:
movie = (nda.Stimulus & scan_key).fetch1('movie') # stimulus images synchronized with nda.FrameTimes
movie.shape #(height x width x frames)

In [ ]:
movie_times = (nda.FrameTimes() & scan_key).fetch1('frame_times') # timestamps of each image in the stimulus
movie_times.shape

## Trials


A trial represents a short segment of the stimulus. Trial information is logged in `nda.Trial` and indexed by `trial_idx`. 

There are three types of trials: `Clip`, `Monet2`, and `Trippy`. 

`Clip` types show segments that contain high-entropy scenes such as Hollywood clips, sports clips and rendered movies. 

`Monet2` and `Trippy` types show parametric segments. 

Each stimulus segment has a unique identifier in its `condition_hash` and are indexed in the tables `nda.Clip`, `nda.Monet2`, and `nda.Trippy`. 

`start_idx` and `stop_idx` are the indices of the movie in `nda.Stimulus` during which the trial was ongoing. 

`start_frame_time` and `end_frame_time` are the timestamps of the `start_idx` and `end_idx` in seconds relative to the start of the scan. 

`frame_times` are the timestamps of every frame shown in the movie segment at the original presentation frequency (not synced to the scan). 

*Note that in `nda.Trial`, `frame_times` differs the attribute `frame_times` in `nda.FrameTimes`. Nomenclature will be updated and clarified in a future version.*

In [ ]:
nda.Trial & scan_key

A stimulus condition (the *type* column) is a unit of movie stimulus, and a stimulus trial (the *trial_idx* column) is an instance of that condition being shown to the animal, as well as the activity and behavior that are recorded during that period.

Detailed information about the stimulus during each trial can be obtained by restricting into the appropriate type-specific table. The example trial_key below restricts to a `Monet2` type trial. 

In [ ]:
trial_key = {'session': 4, 'scan_idx': 7, 'trial_idx': 8}

In [ ]:
trial_info = nda.Trial & trial_key
trial_info

[Join](https://docs.datajoint.io/python/queries/07-Join.html) `trial_info` with `nda.Monet2` using the `*` operator to get all available information about the stimulus during that trial. 

`Monet2` is a directional stimuli, and the vector of directions during the trial is stored in the `directions` attribute. See technical methods for a more detailed description of the contents of this and other stimulus type tables. 

In [ ]:
trial_info * nda.Monet2

Use `start_idx` and `end_idx` to view the part of the stimulus corresponding to the trial. These essentially give you the matched frames from nda.FreameTimes that are matched to a specific image in the movie in nda.Stimulus

In [ ]:
start, end = (trial_info * nda.Monet2).fetch1('start_idx', 'end_idx') # Fetch indices of trial
print(f'Trial starts at index: {start} and ends at index {end}')

In [ ]:
stimulus_trial_slice = movie[:,:,slice(start, end)] # slice movie according to indices of trial
stimulus_trial_slice.shape

In [ ]:
fig, axs = plt.subplots(1, 2, dpi=150) # view first and last frame of movie slice
axs[0].imshow(stimulus_trial_slice[:,:,0], cmap='gray')
axs[0].set_title(f'frame: {start}')
axs[1].imshow(stimulus_trial_slice[:,:,-1], cmap='gray')
axs[1].set_title(f'frame: {end}')
[ax.axis('off') for ax in axs];

# Spike traces

Spike traces are in `nda.Activity` and associated with single units (`unit_id's` which correspond to a specific neuron from CAVE client functonally matched table).

In [ ]:
unit_key = {'session': 4, 'scan_idx': 7, 'unit_id': 1262}

In [ ]:
nda.Activity() & unit_key

In [ ]:
spike_trace = (nda.Activity() & unit_key).fetch1('trace')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3), dpi=150)
ax.plot(spike_trace, c='k')
ax.set_xlabel('Frames')
ax.set_ylabel('Spike trace')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

You can see that on the x axis we have the frames because this shows the activity of the neurone to the whole movie shown. If you wanted the specific spike trace activity of the neurone for a specific stimulus you can subset the graph by only containingthe frames for which the specific stimulus was shown. Let's do that for one trial of the 'Trippy' stimulus

In [ ]:
#Picking a trial instance of Trippy'
trip = {'session': 4, 'scan_idx': 7, 'trial_idx':400}
trial_trip=  nda.Trial & trip
trial_trip

In [ ]:
#Etxracting start and end frames
start, end = trial_trip.fetch1('start_idx', 'end_idx') # Fetch indices of trial
print(f'Trial starts at index: {start} and ends at index {end}')

In [ ]:
#Subsetting teh spike trace of the neurone we visualised it for earlier
fig, ax = plt.subplots(figsize=(10, 3), dpi=150)
ax.plot(spike_trace[start:end], c='k')
ax.set_xlabel('Frames')
ax.set_ylabel('Spike trace')
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_title('Spike trace of neuron 1262 for trial 400 (an instance of the Trippy trial)')
plt.show()

## View  fluorescence trace, and spike trace for the same unit

Use `nda.ScanUnit` to bridge between `unit_id` and `mask_id` to plot both the calcium trace spike trace and for a single unit.

In [ ]:
nda.Fluorescence & (nda.ScanUnit & unit_key)

In [ ]:
calcium_trace = (nda.Fluorescence & (nda.ScanUnit & unit_key)).fetch1('trace')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3), dpi=150)
ax.plot(calcium_trace/ np.max(calcium_trace), c='g', alpha=0.5, label='calcium')
ax.plot(spike_trace/ np.max(spike_trace), c='k', label='spike', alpha=0.5)
ax.set_xlabel('Frames')
ax.legend()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Selecting relevant stimuli for our analysis


It is important that we are able to analyse the relationship between the reponse of different neurons to the same stimuli. While the Trippy and Monet stimuli where randomised across different scans and sessions, a particular subsed of six stimuli the Clippy class (the natural stimuli) where always the same. In addition, these stimuli where also repeated 10 times per scan, allowing to also understand the reliability of the trace response of the neurons to the same stimuli. This allows to both identify neurons that do not have excessive trial repeat variability, but also that do not have excessive habituation to a stimuli.

The analysis of the neuronal activity that is presented in this project will focus exclusively on the repsonses to this subset of stimuli, hencefort referred to as 'Oracle stimuli'. 

## Identifying and extracting Oracle stimuli

In [ ]:
#session and scan values for which we have cells
pairs = {(4, 7), (5, 3), (5, 6),(5, 7),(6, 2),(6, 4),(6, 6),(6, 7),(7, 3),(7, 4),(7, 5),(8, 5),(9, 3),(9, 4),(9, 6)}

In [ ]:
def stimulus_idx_extractor(pairs, stimulus_type, only_oracle = False):
    '''This function extracts all the indices for a certain session and scan idx that correspond to a
     specific stimulus type in th MICrONS dataset
    
    Parameters:
    pairs: list of tuples specifying (session, scan_idx)
    stimulus_type: string specifying stimulus type, one of 
        'stimulus.Clip': natural movies, 
        'stimulus.Trippy': Trippy type stimuli,
        'stimulus.Monet2': Monet type stimuli
    only_oracel: Boolean to specify if only want the oracle stimuli out of the natural stimuli, this option
    only works if you specify stimulus_type = 'stimulus.Clip'
    
    Returns:
    dictionary of lists where key is (session, scan_idx) tuple and values is a list of lists 
    where each sub list is [start_idx, end_idx] of indices containing the relevant activity or movie frames
    for that stimulus repeat
    '''
    #Extract all hashes identifying unique stimuli 
    set_list = []#define set list

    #loop through relevant sessions and scans
    for i in tqdm(pairs): 
        #define keys to query database, in this case selects only clip stimuli (which contains oracol subset)
        trial_id = {'session':i[0], 'scan_idx':i[1],'type':stimulus_type}

        #find condition hashes of all stimuli in the key and save in a set to remove duplicates
        s = set((nda.Trial&trial_id).fetch('condition_hash'))

        #Append this set to a list of set of hashes
        set_list.append(s)

    #Use the * operator to unpack each set in the list and find intersection between all sets to find all
    #unique hashes
    res = set.intersection(*set_list)
    
    if only_oracle == True:
        oracle = []
        #loop through all hashes
        for i in tqdm(res):
            #query a generic session and scan
            test_id1 = {'session':7, 'scan_idx':3,'condition_hash':i}

            #If hash contained ten times its an oracle stimuli so save the hash
            if len(nda.Trial&test_id1) == 10:
                oracle.append(i)
        res = oracle
    seq_oracle = defaultdict(list) #Initiate dictionary

    
    for i in tqdm(sorted(res)):
        #Select a hash of a trial and extract all the relevant identifiers across all sessions and scans
        sess, scid, startid, endid = (nda.Trial&{'condition_hash':i}).fetch('session','scan_idx', 'start_idx', 'end_idx')
        
        #for each entry save the start and end of each occurrance of the stimulus specified by the hash
        #segmenting by the relevant session and scan id
        for s, sc, start, end in zip(sess, scid, startid, endid):
            if (s,sc) in pairs:
                seq_oracle[(s, sc)] += [[start, end]]
            else:
                continue
    return seq_oracle

In [ ]:
seq_oracle = stimulus_idx_extractor(pairs, 'stimulus.Clip', only_oracle = True)

## Checking correlation between same stimuli across sessions

Here we do a quick manual double check to see if the movie stimuli for one of the oracle trials between two difference sessions and scans are indeed the same stimulus

In [ ]:
#Select same oracel stimulus hash across two sessions and scans
test_id1 = {'session':4, 'scan_idx':7,'condition_hash':list(oracle)[0]}
test_id2 = {'session':5, 'scan_idx':3,'condition_hash':list(oracle)[0]}

In [ ]:
#Fetch the ids of the specified stimuli
ses1 = (nda.Trial&test_id1).fetch('trial_idx')
ses2 = (nda.Trial&test_id2).fetch('trial_idx')

In [ ]:
#Extract movie
mov1 = (nda.Stimulus & {'session':4, 'scan_idx':7}).fetch1('movie')

In [ ]:
# Select two different trials of the same stimulus within the same session
#Trial one
trial_id = {'session':4, 'scan_idx':7,'trial_idx':ses1[0]}
st = (nda.Trial&trial_id).fetch1('start_idx', 'end_idx')
t1 = mov1[:,:,slice(st[0], st[1])]

#Trial two
trial_id2 = {'session':4, 'scan_idx':7,'trial_idx':ses1[1]}
st2 = (nda.Trial&trial_id2).fetch1('start_idx', 'end_idx')
t2 = mov1[:,:,slice(st2[0], st2[1])]

del mov1
gc.collect()

In [ ]:
#Extract movie of the second stimulus
mov2 = (nda.Stimulus & {'session':5, 'scan_idx':3}).fetch1('movie')

In [ ]:
# Select two different trials of the second stimulus within the same session
#Trial one
trial_id3 = {'session':5, 'scan_idx':3,'trial_idx':ses2[4]}
st3 = (nda.Trial&trial_id3).fetch1('start_idx', 'end_idx')
t3 = mov2[:,:,slice(st3[0], st3[1])]

#Trial two
trial_id4 = {'session':5, 'scan_idx':3,'trial_idx':ses2[1]}
st4 = (nda.Trial&trial_id4).fetch1('start_idx', 'end_idx')
t4 = mov2[:,:,slice(st4[0], st4[1])]

In [ ]:
#Visulalise a frame of the first session
plt.imshow(t1[:,:,-1])

In [ ]:
#Visualise a frame if the other session
plt.imshow(t2[:,:,-1])

In [ ]:
#Calulate the correlation between the matrices
np.corrcoef(np.array((t1[:,:,:].ravel(), t2[:,:,:].ravel(),t3[:,:,:].ravel(), t4[:,:,:].ravel())))

These results highlight high correlations between frames of the natural image stimulus that are indexed by the same hash across sessions and scans, suggesting that they are similar enough to calculate the correlation of spike trace activity on.

## Extracting and saving relevant spike traces for Oracle stimuli and reliability threshold

In this section we extract both the spike traces for teh repeated oracle stimuli and also calculate the ccmax measure to assess the reliability of neuron repsonse

In [5]:
func_locations = pd.read_csv('clean_functional_locations.csv')
func_locations.head(2)

In [ ]:
def regularizer_calc(unit_key, calcium = False):
    '''This function extracts the activity of each neuron at each of the 10 Oracle trials
    across the 6 stimuli
    
    Parameters:
    unit_key: dictionary containing session, scan_idx and unit_id to identify the cell
    calcium: boolean deciding wether to extract activity as the calcium trace or the spike trace
    
    Returns:
    Array with nueron activity, with one row per repeat of the oracle stimuli
    '''
    
    if calcium == True:
        #Query calcium trace from data joint 
        st1 = (nda.Fluorescence & (nda.ScanUnit & unit_key)).fetch1('trace')
    else:
        #Query spike trace from data joint 
        st1 = (nda.Activity& unit_key).fetch1('trace')
    
    ys = []
    
    for s,e in seq_oracle[(unit_key['session'], unit_key['scan_idx'])]:
        #extract activity for each of the oracle stimulus in the specific session and scan
        # crop to 62 since not all of the trials had 63 frames exactly and the shortest length was 62
        ys+=list(st1[s:s+62])
 
    #This returns an array where each row is all of the activity across the 10 repeats for one of the
    #6 oracle stimuli
    six_oracles = np.array(ys).reshape((6, 620))
    
    #Turning the matrix in one where each row is a repeat of all the oracle stimuli
    repeats = []
    for i in range(0,620,62):
        # select a repeat of all six oracle stimuli
        repeats.append(six_oracles[:, i:i+62].ravel())
    
        
    return np.array(repeats)

In [ ]:
def activity_extractor(unit_key, seq_dict, calcium = False):
    
    '''This function extracts the activity of each neuron at each of the intervals specified in the
    seq_dict parameters
    
    Parameters:
    unit_key: dictionary containing session, scan_idx and unit_id to identify the cell
    calcium: boolean deciding wether to extract activity as the calcium trace or the spike trace
    seq_dict: dictionary of lists where key is (session, scan_idx) tuple and values is a list of lists 
    where each sub list is [start_idx, end_idx] of indices containing the relevant activity or movie frames
    for that stimulus repeat
    
    Returns:
    Array with nueron activity, with one row per stimuli
    '''
    
    if calcium == True:
        #Query calcium trace from data joint 
        st1 = (nda.Fluorescence & (nda.ScanUnit & unit_key)).fetch1('trace')
    else:
        #Query spike trace from data joint 
        st1 = (nda.Activity& unit_key).fetch1('trace')
    
    ys = []
    
    for s,e in seq_dict[(unit_key['session'], unit_key['scan_idx'])]:
        #extract activity for each of the stimulus in the specific session and scan
        # crop to 62 since not all of the trials had 63 frames exactly and the shortest length was 62
        ys.append(list(st1[s:s+62]))
 
   
    
        
    return np.array(ys)

In [ ]:
def ccmax(ys):
    '''This function calculates the cc max measure described in Ding et al. 2023'''
    var_ymean = np.var(np.mean(ys, axis = 0))
    m_var_ys = np.mean([np.var(i) for i in ys])

    numer = (len(ys)*var_ymean)-m_var_ys
    denom = (len(ys)-1)*var_ymean
    return np.sqrt(numer/denom)
    

In [ ]:
warnings.filterwarnings("ignore")
ccmaxs = [] #save ccmax values in a list
st = [] #save spike traces

for i in tqdm(range(len(func_locations))):
    
    #Identify session, scan and unit number
    sess = func_locations['session'].iloc[i]
    scan = func_locations['scan_idx'].iloc[i]
    unit = func_locations['unit_id'].iloc[i]
    
    #Generate relevant unit key
    unit_key = {'session':sess, 'scan_idx':scan,'unit_id':unit}
    
    #Extract matrix of relevant activity traces
    ys = regularizer_calc(unit_key, calcium = True)
    
    #Calculate ccmax for the specific neuron and save it in the list
    ccmaxs.append(ccmax(ys))
    
    #Caluclate mean activation column and append it to activations
    mean = np.mean(ys, axis = 0)
    full = np.vstack((ys, mean))
    st.append(full)
    
    del ys
    del sess
    del scan
    del unit
    del mean
    del full
    gc.collect()

In [ ]:
#Add it as a fetaure column to existing functionally matched data
func_locations['ccmax'] = ccmaxs
#modify name according to type of activity you are extracting
func_locations['oracle_calcium_traces'] = st

In [ ]:
# Saving the data
#If spike traces
#func_locations.to_pickle('clean_functional_ccmax_ost.pkl')

#If calcium traces
#func_locations.to_pickle('clean_functional_ccmax_calcium.pkl')

## Extracting Relevant Movie Stimulus sections

It is known that once exposed to a repeated stimuli a neurone habituates and responses are suppressed over time. In order to estimate receptive fields using the stimuli we will utilise only the first repeat of the six oracle stimuli of each neurone, which should be the one for wich neuron response is more strongly characterised. 

Since for some of the computations that we will have to overtake at present the data joint server crashes, we will (like we did for spike traces), save the relevant movie sections for each session and scan so as to use it locally (if enough computing power is available) or on a different server.

In [ ]:
def first_mov_extractor(pair):
    #generate key and query movie
    movie_key = {'session': pair[0], 'scan_idx': pair[1]}
    mov = (nda.Stimulus&movie_key).fetch1('movie')
    
    #Select the index of the intervals of the first set of oracle stimuli (6 index, one for every stimuli)
    intervals = np.array(seq_oracle[(pair[0],pair[1])])[[*range(0,60, 10)]]
    
    #extract movie segments
    m = mov[:,:, intervals[0][0]:intervals[0][0]+62]
    for p in intervals[1:]:
        m1 = mov[:,:, p[0]:p[0]+62]
        m = np.dstack((m,m1))
    return m

In [ ]:
# The data joint serverkernel keeps on crashing so once the kernel restarts add the pairs you have extracted so far
# to the 'to_rem' list so that you can continue with the ones left
pairs_new = list(pairs.copy())
to_rem = [(6, 2), (7, 4), (9, 3), (4, 7), (6, 6), (6, 4), (5, 7), (7, 3), (8, 5), (6, 7),
(5, 6), (5, 3), (7, 5), (9, 4)]

for i in to_rem:
    pairs_new.remove(i)

In [ ]:
# STILL NEED TO EXTRACT (9, 6)!!!

In [ ]:
for pair in pairs_new:
    movie = first_mov_extractor(pair)
    with open(f'm_{pair[0]}_{pair[1]}.pkl', 'wb') as file:
        pickle.dump(movie, file)
    
    del movie
    gc.collect()

In [ ]:
#To import the file once needed:
#with open(f'm_{4}_{7}.pkl', 'rb') as file:
#    mt = pickle.load(file)

NOTE:
at present data joint seems to be unable to handle the import of the movie for session 9 scan 6 (i.e. (9,6)). It was thus not possible to extract and save this movie for further receptive field analysis